## Import Libraries

In [8]:
import numpy as np
import os
import pickle
import csv
import math
from gensim.models import Word2Vec
from sklearn.utils import resample
import random
import time

random.seed(3919)

In [9]:
import pandas as pd
import re
from collections import Counter

In [10]:
from pprint import pprint
import gensim
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import gensim.test.utils 
import itertools
from sklearn.utils import resample
from multiprocessing import Pool

In [11]:
%matplotlib inline
import matplotlib.pyplot as plt

In [12]:
#read df phrased lemma with preprocessed text (from office PC)
df = pd.read_pickle('/Users/adrianacupp/Desktop/GitHub/russia_lio_contestation/corpus_president_of_russia/df_lemma_preprocessing.pkl')

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6779 entries, 0 to 4311
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   date           6779 non-null   object
 1   url            6779 non-null   object
 2   title          6779 non-null   object
 3   speaker        6779 non-null   object
 4   text           6779 non-null   object
 5   text_clean     6779 non-null   object
 6   word_count     6779 non-null   int64 
 7   lemma          6779 non-null   object
 8   phrased_lemma  6779 non-null   object
dtypes: int64(1), object(8)
memory usage: 529.6+ KB


In [13]:
data_lemma = df['phrased_lemma'].tolist()

In [14]:
#dictionary
id2word = corpora.Dictionary(data_lemma)
#corpus
texts = data_lemma
#term document matrix
corpus = [id2word.doc2bow(text) for text in texts]

In [15]:
[[(id2word[i], freq) for i, freq in doc] for doc in corpus[:2]]

[[('1_январь', 1),
  ('2006_год', 1),
  ('86', 1),
  ('8_процент', 1),
  ('абсолютно_уверенный', 1),
  ('административный_реформа', 1),
  ('администрация_правительство', 1),
  ('бюджетный_сфера', 1),
  ('важный_составлять', 1),
  ('введение', 1),
  ('взаимодействие', 1),
  ('вид', 1),
  ('власть', 7),
  ('вместе', 1),
  ('внести', 1),
  ('внутриполитический_ситуация', 1),
  ('возлагаться', 1),
  ('вопрос', 2),
  ('вопрос_вопрос', 1),
  ('вопрос_ставить', 1),
  ('время', 1),
  ('вступать_сила', 1),
  ('вступительный_слово', 1),
  ('входить', 1),
  ('выборный_процесс', 1),
  ('выплата_компенсация', 1),
  ('вырабатывался', 1),
  ('высокий', 2),
  ('говорить_знать', 1),
  ('говорить_человек', 1),
  ('год', 1),
  ('государство_общество', 1),
  ('гражданин', 1),
  ('далёкий_работать', 1),
  ('два', 1),
  ('два_слово', 1),
  ('дело', 1),
  ('дело_государство', 1),
  ('день', 1),
  ('депутат_государственный', 1),
  ('десять', 1),
  ('деятельность', 1),
  ('длительный', 1),
  ('добрый_день', 1)

In [16]:
print('Number of unique tokens: %d' % len(id2word))
print('Number of documents: %d' % len(corpus))

Number of unique tokens: 105746
Number of documents: 6779


## Training the LDA

### num_of_topics=15 

In [29]:
#TRAINING THE MODEL (num_of_topics=10)
num_topics = 15
chunksize = 100
passes = 10 #controls how often we train the model on the entire corpus.
iterations = 200
eval_every = None  # Don't evaluate model perplexity, takes too much time.

In [30]:
lda_model_15 = gensim.models.LdaMulticore(corpus = corpus,
                                       id2word = id2word,
                                       num_topics = num_topics,
                                       random_state = 42,
                                       chunksize = chunksize,
                                       passes = passes,
                                       per_word_topics=True,
                                       minimum_probability = 0)

In [31]:
pprint(lda_model_15.print_topics())
doc_lda = lda_model_10[corpus]

[(0,
  '0.007*"обсе" + 0.006*"вопрос" + 0.005*"россия" + 0.004*"грузия" + '
  '0.004*"южный_осетия" + 0.003*"абхазия" + 0.003*"говорить" + 0.003*"сказать" '
  '+ 0.003*"совет_европа" + 0.003*"отношение"'),
 (1,
  '0.007*"россия" + 0.004*"страна" + 0.004*"мир" + 0.003*"государство" + '
  '0.003*"безопасность" + 0.003*"отношение" + 0.003*"год" + 0.003*"вопрос" + '
  '0.003*"проблема" + 0.002*"оон"'),
 (2,
  '0.005*"россия" + 0.005*"страна" + 0.003*"год" + 0.003*"регион" + '
  '0.003*"государство" + 0.003*"работа" + 0.003*"развитие" + 0.003*"сегодня" + '
  '0.002*"организация" + 0.002*"мир"'),
 (3,
  '0.003*"россия" + 0.003*"страна" + 0.002*"вопрос" + '
  '0.001*"президент_финляндия" + 0.001*"проблема" + 0.001*"дело" + 0.001*"год" '
  '+ 0.001*"говорить" + 0.001*"государство" + 0.001*"знать"'),
 (4,
  '0.007*"россия" + 0.005*"соотечественник" + 0.004*"страна" + 0.004*"церковь" '
  '+ 0.003*"год" + 0.003*"рубеж" + 0.003*"российский" + 0.003*"работа" + '
  '0.002*"сегодня" + 0.002*"русский_

In [32]:
#u_mass
coherence_model_lda_15 = CoherenceModel(model=lda_model_15, corpus=corpus, coherence='u_mass')
coherence_lda_10 = coherence_model_lda_15.get_coherence()
print('\nCoherence Score: ', coherence_lda_10)


Coherence Score:  -1.1288381339570825


In [33]:
#c_v
coherence_model_lda_15 = CoherenceModel(model=lda_model_15, texts=texts, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda_15.get_coherence()
print('nCoherence Score: ', coherence_lda)

nCoherence Score:  0.32856659383655507


In [24]:
#save model in pickle format to working directory
#lda_model_15.save("wordembedding/models/lda_model_10.pkl")

In [ ]:
#load model back into your workspace from working directory
lda_15 = gensim.models.LdaModel.load("wordembedding/models/lda_model_10.pkl")

### visualize topic models

In [25]:
from imp import reload
import pyLDAvis
import pyLDAvis.gensim_models

In [26]:
pyLDAvis.enable_notebook()

In [34]:
from imp import reload
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
pyLDAvis.gensim_models.prepare(lda_model_10, corpus, id2word)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
10     0.028745  0.088147       1        1  23.640804
8      0.105143 -0.014531       2        1  21.143532
12     0.118835 -0.085271       3        1  18.024615
9      0.110653 -0.058138       4        1  15.062769
1      0.082429 -0.027184       5        1   7.976029
2      0.061829  0.061628       6        1   5.652248
7     -0.034914  0.209279       7        1   3.260174
0      0.046233 -0.040167       8        1   2.472863
4      0.014194  0.090085       9        1   1.986784
13    -0.050589 -0.020105      10        1   0.269014
6     -0.151427 -0.063198      11        1   0.230912
11    -0.094059 -0.014663      12        1   0.098860
14    -0.074815 -0.045106      13        1   0.080284
3     -0.082774 -0.034887      14        1   0.054186
5     -0.079484 -0.045890      15        1   0.046926, topic_info=          Term          Freq         Total Category  logprob  loglift
17      вопрос  15334.000000  15334.000000  Default  30.0000  30.0000
177     россия  18699.000000  18699.000000  Default  29.0000  29.0000
738     страна  15760.000000  15760.000000  Default  28.0000  28.0000
504    сказать  10340.000000  10340.000000  Default  27.0000  27.0000
279   говорить  11704.000000  11704.000000  Default  26.0000  26.0000
...        ...           ...           ...      ...      ...      ...
570   являться      1.155140   3859.164095  Topic15  -7.2982  -0.4496
2722   партнёр      1.140233   3689.261437  Topic15  -7.3112  -0.4176
181    сделать      1.170005   6087.247965  Topic15  -7.2854  -0.8926
182    сегодня      1.163469   6777.028572  Topic15  -7.2911  -1.0055
1982   считать      1.146596   5103.392287  Topic15  -7.3057  -0.7365

[1372 rows x 6 columns], token_table=       Topic      Freq                      Term
term                                            
3123       1  0.940805                10_процент
3123       5  0.009224                10_процент
3123       6  0.027671                10_процент
3123       9  0.009224                10_процент
46965      1  0.958354                15_процент
...      ...       ...                       ...
3696      10  0.001046                      язык
3696      11  0.001046                      язык
81236      2  0.976438  языковой_образовательный
78194      4  0.967205                    ямайка
80842      2  0.965908      януковичем_оппозиция

[5547 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[11, 9, 13, 10, 2, 3, 8, 1, 5, 14, 7, 12, 15, 4, 6])